In [2]:
with open('./datasets/animals/translate.py') as f :
    content = f.readline()
    
    # eval - > 문자열 안에 있는 것을 인식 해주는 함수 
    contents1 = eval(content[content.index('{'):content.index('}') + 1])
    
    # contents.items() <-- items는 한 쌍 씩 갖고옴
    # key , value 순서를 바꾸는 로직 
    contents2 = {v:k for k, v in contents1.items()}
    
print(contents1, contents2, sep='\n')

{'cane': 'dog', 'cavallo': 'horse', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'pecora': 'sheep', 'scoiattolo': 'squirrel', 'dog': 'cane', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'spider': 'ragno', 'squirrel': 'scoiattolo'}
{'dog': 'cane', 'horse': 'cavallo', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'sheep': 'pecora', 'squirrel': 'scoiattolo', 'cane': 'dog', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'ragno': 'spider', 'scoiattolo': 'squirrel'}


In [5]:
from glob import glob
import os
root = './datasets/animals/original/' 

directories = glob(os.path.join(root, '*'))

for directory in directories :
    try : 
        os.rename(directory, os.path.join(root, contents1[directory[directory.rindex('\\') + 1:]]))
    except KeyError as e:
        os.rename(directory, os.path.join(root, contents2[directory[directory.rindex('\\') + 1:]]))
    
# animals-->original

In [7]:
root = './datasets/animals/original/' 

directories = glob(os.path.join(root, '*'))

print(directories)
directory_names = []

for directory in directories :
    directory_names.append(directory[directory.rindex('\\') + 1:])
print(directory_names)

['./datasets/animals/original\\butterfly', './datasets/animals/original\\cat', './datasets/animals/original\\chicken', './datasets/animals/original\\cow', './datasets/animals/original\\dog', './datasets/animals/original\\elephant', './datasets/animals/original\\horse', './datasets/animals/original\\sheep', './datasets/animals/original\\spider', './datasets/animals/original\\squirrel']
['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']


In [8]:
# 파일명 변경 로직
root = './datasets/animals/original/' 

for name in directory_names :
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))) :
        old_file = os.path.join(root + name + '/', file_name)
        new_file = os.path.join(root + name + '/', name + str(i+1) + '.png')

        os.rename(old_file, new_file)

In [9]:
# 타겟 가져오기 로직
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1./255)

generator = image_data_generator.flow_from_directory(root, target_size=(150, 150), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 26179 images belonging to 10 classes.
{'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [15]:
import pandas as pd 

a_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets': generator.classes})
a_df

,file_paths,targets
0,./datasets/animals/original/butterfly\butterfl...,0
1,./datasets/animals/original/butterfly\butterfl...,0
2,./datasets/animals/original/butterfly\butterfl...,0
3,./datasets/animals/original/butterfly\butterfl...,0
4,./datasets/animals/original/butterfly\butterfl...,0
...,...,...
26174,./datasets/animals/original/squirrel\squirrel9...,9
26175,./datasets/animals/original/squirrel\squirrel9...,9
26176,./datasets/animals/original/squirrel\squirrel9...,9
26177,./datasets/animals/original/squirrel\squirrel9...,9


In [16]:
a_df.loc[:, 'file_paths'] = a_df.file_paths.apply(lambda x: x.replace('\\', '/'))

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(a_df.file_paths, a_df.targets, stratify=a_df.targets, test_size=0.2, random_state=124)
print(y_train.value_counts())
print(y_test.value_counts())

targets
4    3890
8    3857
2    2478
6    2098
0    1690
3    1493
9    1490
7    1456
1    1334
5    1157
Name: count, dtype: int64
targets
4    973
8    964
2    620
6    525
0    422
3    373
9    372
7    364
1    334
5    289
Name: count, dtype: int64


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=124)
print(y_val.value_counts())

targets
4    623
8    617
2    397
6    336
0    270
3    239
9    238
7    233
1    213
5    185
Name: count, dtype: int64


In [28]:
import shutil
root = './datasets/animals/'

for file_path in X_train :
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    print(animal_dir)
    destination = os.path.join(root, 'train/' + animal_dir)

    if not os.path.exists(destination) : 
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

chicken
dog
spider
elephant
dog
squirrel
spider
dog
chicken
chicken
spider
cat
spider
dog
elephant
butterfly
spider
horse
spider
butterfly
chicken
dog
elephant
sheep
horse
dog
sheep
elephant
horse
spider
sheep
dog
cow
horse
cow
dog
spider
sheep
chicken
elephant
cat
sheep
horse
cat
dog
horse
spider
spider
chicken
sheep
sheep
butterfly
spider
spider
cow
cow
butterfly
butterfly
chicken
spider
spider
spider
chicken
dog
spider
elephant
squirrel
sheep
sheep
squirrel
squirrel
squirrel
dog
dog
spider
dog
cat
spider
butterfly
horse
dog
dog
horse
dog
chicken
spider
spider
sheep
spider
spider
squirrel
butterfly
butterfly
dog
spider
chicken
butterfly
butterfly
spider
dog
dog
elephant
chicken
cat
cat
squirrel
sheep
spider
horse
dog
spider
spider
spider
spider
sheep
butterfly
cat
chicken
horse
spider
sheep
spider
squirrel
horse
elephant
dog
squirrel
dog
cow
cat
cow
dog
squirrel
horse
dog
dog
horse
spider
butterfly
chicken
spider
butterfly
dog
cat
dog
spider
dog
spider
butterfly
cat
cat
spider
chicke

In [29]:
import shutil
root = './datasets/animals/'

for file_path in X_val :
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    print(animal_dir)
    destination = os.path.join(root, 'validation/' + animal_dir)

    if not os.path.exists(destination) : 
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

elephant
sheep
horse
chicken
horse
dog
dog
dog
spider
chicken
squirrel
dog
butterfly
cow
dog
chicken
chicken
dog
butterfly
sheep
dog
dog
horse
sheep
horse
horse
butterfly
cat
chicken
chicken
elephant
elephant
spider
horse
dog
cow
elephant
spider
horse
dog
sheep
cow
spider
butterfly
sheep
elephant
spider
horse
spider
spider
horse
chicken
cat
squirrel
dog
butterfly
cat
spider
dog
sheep
spider
chicken
dog
chicken
butterfly
butterfly
horse
cow
spider
cow
spider
butterfly
spider
dog
butterfly
dog
spider
dog
squirrel
butterfly
butterfly
spider
spider
chicken
butterfly
cat
elephant
chicken
chicken
spider
sheep
dog
spider
squirrel
spider
squirrel
horse
horse
butterfly
chicken
dog
chicken
horse
horse
sheep
butterfly
cat
spider
chicken
sheep
chicken
sheep
dog
cat
spider
cat
cow
chicken
sheep
chicken
chicken
elephant
dog
squirrel
dog
squirrel
horse
horse
chicken
spider
spider
cow
butterfly
dog
spider
spider
chicken
dog
dog
dog
elephant
spider
elephant
dog
chicken
spider
chicken
spider
horse
cat
c

In [30]:
import shutil
root = './datasets/animals/'

for file_path in X_test :
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    print(animal_dir)
    destination = os.path.join(root, 'test/' + animal_dir)

    if not os.path.exists(destination) : 
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

squirrel
elephant
horse
dog
sheep
spider
cow
spider
cow
butterfly
dog
spider
spider
chicken
elephant
cow
spider
spider
dog
spider
elephant
sheep
spider
dog
dog
butterfly
cow
spider
dog
chicken
sheep
chicken
sheep
cow
butterfly
butterfly
dog
dog
dog
squirrel
dog
sheep
horse
butterfly
horse
spider
chicken
dog
squirrel
spider
spider
spider
butterfly
dog
elephant
horse
butterfly
chicken
squirrel
elephant
butterfly
elephant
chicken
spider
squirrel
spider
butterfly
cat
sheep
chicken
dog
spider
squirrel
dog
butterfly
spider
elephant
cat
cow
spider
butterfly
horse
horse
spider
dog
dog
dog
sheep
cow
horse
butterfly
dog
dog
dog
spider
dog
butterfly
spider
horse
chicken
chicken
spider
spider
dog
sheep
spider
dog
cow
elephant
squirrel
spider
horse
squirrel
butterfly
dog
chicken
spider
sheep
elephant
dog
cow
dog
spider
dog
dog
sheep
horse
horse
dog
sheep
chicken
cat
horse
chicken
butterfly
spider
cat
dog
horse
butterfly
spider
butterfly
cow
butterfly
horse
elephant
squirrel
horse
spider
chicken
cat